In [ ]:
from itertools import cycle, islice, product 

from DyClee.algorithms import * 
import warnings
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from copy import deepcopy

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_blobs

In [ ]:
# function to return Xmin and ymin for rectangle parameters
def hyperbox_coordinates(uC, sizes):     
    center = uC.get_center() 
    Xmin = (center[0] - sizes[0]/2)
    ymin = (center[1] - sizes[1]/2)
    return (Xmin, ymin)

In [ ]:
# Make moons data
np.random.seed(0)
n_samples = 1500
colors = np.array(['#377eb8', '#ff7f00'])
X,y = make_circles(n_samples, factor=.5, noise=.05)
X, y = make_moons(n_samples, noise=.05)
X = np.random.rand(n_samples, 2)
X = np.random.rand(n_samples, 2)
X = StandardScaler().fit_transform(X)
plt.scatter(X[:,0], X[:,1], s=10, color=colors[0])
plt.xlim(-2.5, 2.5)
plt.ylim(-2.5, 2.5)
# plt.gca().set_aspect('equal')

In [ ]:
# Context matrix
context = np.vstack([X.min(axis=0), X.max(axis=0)])

In [ ]:
# Create dyclee class object 
dyclee = SerialDyClee(phi=0.06, context=context)

In [ ]:
%%time

# Run dyclee
#dyclee.run_dataset(data=X, targetcol=y)
dyclee.run_dataset(data=X)

In [ ]:
# combine all clusters 
all_uC = dyclee.A_list + dyclee.O_list

print('There are',len(all_uC),'microclusters')
print('Dense & Semi-dense total:',len(dyclee.A_list))
print('Low dense total:', len(dyclee.O_list))

In [ ]:
# Plot hyperboxes 

# create dataframe
cluster_df = pd.DataFrame([uC.get_center() for uC in all_uC], columns=['x', 'y'])

# hyperbox sizes 
sizes = dyclee._get_hyperbox_sizes()


print("the hyperbox size is: ", sizes)
# plot microclusters
sns.scatterplot(x='x',y='y',data=cluster_df)
plt.gca().set_aspect('equal')

# plot hyperboxes
for uC in all_uC: 
    xy = hyperbox_coordinates(uC, sizes)
    plt.gca().add_patch(Rectangle(xy,sizes[0],sizes[1],linewidth=0.5,edgecolor='r',facecolor='none', clip_on=False))


In [ ]:
# Plot density  

# Graph legend  
# Dense = 0
# Semi-Dense = 1
# Low-Dense = 2

# create dataframe       
cluster_df = pd.DataFrame([np.append(uC.get_center(), np.array([uC.density_type]), 0) for uC in all_uC], columns=['x', 'y', 'z'])

# Change string values to ints
cluster_df.loc[(cluster_df.z == 'Dense'),'z']= int(0)
cluster_df.loc[(cluster_df.z == 'Semi-Dense'),'z']=int(1)
cluster_df.loc[(cluster_df.z == 'Low-Density'),'z']=int(2)

# Re-format column types 
cluster_df = cluster_df.infer_objects()
cluster_df['x'] = cluster_df['x'].astype(float)
cluster_df['y'] = cluster_df['y'].astype(float)
cluster_df['z'] = cluster_df['z'].astype(np.int64)

# plot microclusters
sns.scatterplot(x='x',y='y',hue="z",data=cluster_df)
plt.gca().set_aspect('equal')

# plot hyberboxes
# hyperbox sizes 
#sizes = dyclee._get_hyperbox_sizes()
#print("the hyperbox size is: ", sizes)
#for uC in all_uC: 
#    xy = hyperbox_coordinates(uC, sizes)
#    plt.gca().add_patch(Rectangle(xy,sizes[0],sizes[1],linewidth=0.5,edgecolor='r',facecolor='none', clip_on=False))


In [ ]:
# plot snapshots

snapshot_ordered = {}

for order, timestamps in dyclee.snapshots.items(): 
    for timestamp, cluster_lists in timestamps.items(): 
        snapshot_ordered[timestamp] = cluster_lists
        
timestamp_order = sorted(snapshot_ordered) 

fig, axes = plt.subplots(len(timestamp_order), 2, sharex='col', sharey='row', figsize=(20,600))
cols = ["FinalClusters", "MicroClusters"]
for ax, col in zip(axes[0], cols): ax.set_title(col)

for i, t in enumerate(timestamp_order): 
    final_list = snapshot_ordered[t]['final']
    micro_list = snapshot_ordered[t]['all']

    # plot final cluster
    final_df = pd.DataFrame([np.append(uC.center, [uC.label], 0) for uC in final_list], columns=['x', 'y', 'class'])
    sns.scatterplot(ax=axes[i][0], x='x',y='y',hue='class',data=final_df).legend(loc='lower left', bbox_to_anchor=(1.05,0), ncol=1)
    axes[i][0].set_ylabel(t, rotation=0, size='xx-large', weight='bold')
    
    # plot micro clusters 
    micro_df = pd.DataFrame([np.append(uC.get_center(), [uC.Classk], 0) for uC in micro_list], columns=['x', 'y', 'class'])
    sns.scatterplot(ax=axes[i][1], x='x',y='y',hue='class',data=micro_df).legend(loc='lower left', bbox_to_anchor=(1.05,0), ncol=1)

fig.tight_layout

In [ ]:
# plot snapshots with classes displayed on each point

snapshot_ordered = {}

for order, timestamps in dyclee.snapshots.items(): 
    for timestamp, cluster_lists in timestamps.items(): 
        snapshot_ordered[timestamp] = cluster_lists
        
timestamp_order = sorted(snapshot_ordered) 

fig, axes = plt.subplots(len(timestamp_order), 2, sharex='col', sharey='row', figsize=(20,600))
cols = ["FinalClusters", "MicroClusters"]
for ax, col in zip(axes[0], cols): ax.set_title(col)

for i, t in enumerate(timestamp_order): 
    final_list = snapshot_ordered[t]['final']
    micro_list = snapshot_ordered[t]['all']

    # plot final cluster
    final_df = pd.DataFrame([np.append(uC.center, [uC.label], 0) for uC in final_list], columns=['x', 'y', 'class'])
    f_plot = sns.scatterplot(ax=axes[i][0], x='x',y='y',hue='class',data=final_df)
    f_plot.legend(loc='lower left', bbox_to_anchor=(1.05,0), ncol=1)
    for line in range(0,final_df.shape[0]): 
        f_plot.text(final_df['x'][line], final_df['y'][line], final_df['class'][line], horizontalalignment='left', size='medium', color='black', weight='normal')
    axes[i][0].set_ylabel(t, rotation=0, size='xx-large', weight='bold')
    
    # plot micro clusters 
    micro_df = pd.DataFrame([np.append(uC.get_center(), [uC.Classk], 0) for uC in micro_list], columns=['x', 'y', 'class'])
    m_plot = sns.scatterplot(ax=axes[i][1], x='x',y='y',hue='class',data=micro_df)
    m_plot.legend(loc='lower left', bbox_to_anchor=(1.05,0), ncol=1)
    for line in range(0,micro_df.shape[0]): 
        if micro_df['class'][line] != 'Unclassed': 
            m_plot.text(micro_df['x'][line], micro_df['y'][line], micro_df['class'][line], horizontalalignment='left', size='medium', color='black', weight='normal')
    

fig.tight_layout